In [18]:
import requests
import json
import re
import urllib3
import sqlite3 #数据库操作
import time
import random

In [19]:
#定义保存转发的函数
def bcpl(weibo_id, base_url, headers, Max_page):
    page = 281
    count = 1
    # 当count数量小于预期的number时，进行循环
    while page < Max_page:
        url = base_url + weibo_id + '&page=' + str(page)
        print(url)
        try:  
            urllib3.disable_warnings()
            web_data = requests.get(url, headers=headers,verify=False)    #F12查看data信息
            js_con = web_data.json()    #转换一下数据格式
            # 获取连接下一页评论的max_id
            reposts = js_con['data']['data']    #获得数据中[data]中的[data]
            reposts_data_list = []
            for repost in reposts:    #依次循环获得comments中的数据
                reposts_data = []
                text = repost["text"]     #获得[text]下的数据，也就是评论数据
                label = re.compile(r'</?\w+[^>]*>', re.S)    #删除表情符号
                text = re.sub(label, '', text)    #获得文本评论
                created_at = repost["created_at"] #发布时间
                user = repost["user"]
                user_id = user["id"]
                user_name = user["screen_name"]
                #f.write(text + '\n')    #写入到文件中

                reposts_data.append(text)
                reposts_data.append(created_at)
                reposts_data.append(user_id)
                reposts_data.append(user_name)

                reposts_data_list.append(reposts_data)
                
                print("第" + str(count) + "条转发："+ user_name+" "+text  ) #显示爬取到第几条
                count += 1
            page += 1
            saveData2DB(reposts_data_list,dbpath)
            time.sleep(random.uniform(2,7))
        except Exception as e:
            print("出错了" ,e)
            time.sleep(30)
            continue

In [20]:
def saveData2DB(datalist,dbpath):
    # init_db(dbpath)
    conn = sqlite3.connect(dbpath)
    cursor = conn.cursor()
    sql = "insert into repost(text,created_at,user_id,user_name) values(?,?,?,?)"
    for data in datalist:
        cursor.execute(sql,data)
        conn.commit()
    cursor.close()
    conn.close()

In [21]:
def init_db(dbpath):
    sql = '''
        CREATE TABLE repost
        (
            id INTEGER  PRIMARY KEY AUTOINCREMENT,
            text varchar,
            created_at varchar,
            user_id varchar,
            user_name varchar
        )
    '''
    conn = sqlite3.connect(dbpath) #文件存在即打开，不存在即创建
    cursor = conn.cursor()
    cursor.execute(sql)
    conn.commit()
    conn.close()

In [23]:
dbpath = "J_SNOWMAN_4709336482318969.db"
init_db(dbpath)
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9;',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8,ja;q=0.7,ko;q=0.6',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    'cookie':'_T_WM=91140331655; XSRF-TOKEN=dbaa9e; WEIBOCN_FROM=1110006030; SUB=_2A25MtmPrDeRhGeBM6VcX8C3EyTSIHXVsWQ2jrDV6PUJbkdCOLUH_kW1NRPviX05DLp5DSonfytAJYSxtRicy2lew; SCF=Au501aNiX-mFuWDaJaQT8VIULUm7h7r84iUCNbMmpZkIOxAGsMx3dyAxVJYngpd3GVXUIaGanT8jUPne0yeP9sk.; SSOLoginState=1639060411; MLOGIN=1; M_WEIBOCN_PARAMS=luicode%3D10000011%26lfid%3D102803%26uicode%3D20000174',
    'Referer': 'https://m.weibo.cn',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.55 Safari/537.36',
    'Host': 'm.weibo.cn'
}
#设置user-agent来进行伪装，突破微博反爬限制
base_url = 'https://m.weibo.cn/api/statuses/repostTimeline?id=' # 4709336482318969&page=1
weibo_id = '4709336482318969'  # 要爬取的微博id  #[url=https://m.weibo.cn/detail/4478512314460101]https://m.weibo.cn/detail/4478512314460101[/url]
#打开微博手机端网页[url=https://m.weibo.cn]https://m.weibo.cn[/url]，找到要爬取的微博id！
#手机端网页！手机端网页！手机端网页！
number = 1920 # 设置爬取评论量,爬取量在第X组，爬取时会爬取下来该组的数据，所以最终数据可能会大于number，一般是个整10的数
bcpl(weibo_id, base_url, headers, number)

https://m.weibo.cn/api/statuses/repostTimeline?id=4709336482318969&page=1
第1条转发：中毒多巴胺 大莲哥这大佐味的中文我笑不活了
第2条转发：马群耀和林祎凯要天天开心 啊啊啊小宝说的是中文
第3条转发：snshio 转发微博
https://m.weibo.cn/api/statuses/repostTimeline?id=4709336482318969&page=2
第4条转发：Misach1 中国語〜👏
第5条转发：爱吃可爱多的包子 转发微博
第6条转发：超爱蒲猫猫和韬韬的小白 转发微博
https://m.weibo.cn/api/statuses/repostTimeline?id=4709336482318969&page=3
第7条转发：太子和egg的铲屎官 中文好好哦！
第8条转发：超爱蒲猫猫和韬韬的小白 //@莲理枝AI:中文视频 好有诚意的meme！ 好有诚意的杰尼斯[春游家族]抽个鸟人给我翻译！！！！
第9条转发：Nanwan甜水 强烈要求吻戏！真亲的那种！不要借位的那种！要近景！要两分钟！
第10条转发：脚毛没了 转发微博
第11条转发：zhiaiblt 早meme#目黑莲#
第12条转发：iRis_mini 转发微博
第13条转发：杏菜打嗝 //@杏菜打嗝:什么时候亲嘴啊 评论配图
第14条转发：大雷今天也要加油鸭 赞赞赞
第15条转发：我就要骂个痛快 最终还是打开了组合的知名度
https://m.weibo.cn/api/statuses/repostTimeline?id=4709336482318969&page=4
第16条转发：云帆_宇 转发//@一般般肉:《消えた初恋》、私たちは全部見ました、目黒蓮君の演技はすばらしすぎてね!《snow man 5th》が大ヒットを祝います!《消えた初恋2》はありますか?あるなら、私たちはぜひとても楽しみです!今年はまだ1か月残っている、私たちは目黒蓮君に楽しくて幸せで元気に過ごすことを望みます。
第17条转发：橘勋奇遇记 八嘎帅哥中文说的真好，不用字幕也能听懂[开学季]
第18条转发：随便SHENMEMINGZI 第二季有吗有吗有吗
第19条转发：Zy游啊游 你好可爱啊
第20条转发：zhiaiblt 晚安😘#目黑莲#

https://m.weibo.cn/api/statuses/repostTimeline?id=4709336482318969&page=25
第190条转发：小鱼记忆只五秒o 转发微博
第191条转发：三七啊_ 目黑莲，你好呀
第192条转发：青岛龙的一排一座 你好帅[苦涩]
第193条转发：PP喝烊奶 太可爱了吧讲中文
第194条转发：Sakita_snow Repost
第195条转发：鱼遡 转发微博
https://m.weibo.cn/api/statuses/repostTimeline?id=4709336482318969&page=26
第196条转发：橙雪子 转发微博
第197条转发：yogurtChloee_ 在偷偷看字吧憨憨
第198条转发：me娜丽莎在哪里 第二季gkd
第199条转发：轩的宇霖 消失的初恋yyds
第200条转发：我提着一壶酒来 转发微博
第201条转发：陈宇出警bot 说的很好，下次别说了，(不是)
第202条转发：陈宇出警bot 这中文，尼玛，笑吐了//@莲理枝AI :中文视频 好有诚意的meme！ 好有诚意的杰尼斯[春游家族]抽个鸟人给我翻译！！！！
第203条转发：我是一只小螃蟹_ 我天meme好可爱好可爱好可爱
第204条转发：umi_5 转发微博
https://m.weibo.cn/api/statuses/repostTimeline?id=4709336482318969&page=27
第205条转发：努力赚钱comeon //@深海少女3055:不愧是杰尼斯唯一有中国官博的团，真的诚意满满！meme的中文真的可以，能说这么长一段话祝五单大卖！期待消失的初恋第九集
第206条转发：Z-Qi_L 转发微博
第207条转发：zhiaiblt 早meme#目黑莲#
第208条转发：阿玺加油 早
第209条转发：名字有什么所谓反正乱七八糟 哈哈哈哈哈哈哈哈
第210条转发：什么时候朱白同台 转发微博
第211条转发：林敬Aeriel 转发微博
第212条转发：tako万10 转发微博
第213条转发：piyon216 转发微博
第214条转发：BenBenCharger 等等 这个我竟然没有在首页刷到 怎么会这样！才看到！
https://m.weibo.cn/api/statuses/repostTim

https://m.weibo.cn/api/statuses/repostTimeline?id=4709336482318969&page=49
第398条转发：白西斋 转发微博
第399条转发：·珍珠蚌· 鞥了
第400条转发：Juno_ibkp_ 老公说中文真可爱
第401条转发：nabe_snow 转发微博
第402条转发：賢人80638 转发微博
第403条转发：zl赵大林 转发微博
第404条转发：中毒可乐奶茶爱好者 中文很努力了！
第405条转发：东郭蕴Ou 家人们我不带耳机有些听不懂啊
https://m.weibo.cn/api/statuses/repostTimeline?id=4709336482318969&page=50
第406条转发：zhiaiblt #目黑莲# 早meme
第407条转发：宥憙x321 这这这 这中文说的越来越好了
第408条转发：喻剑威 转发微博
第409条转发：Kio专注培养基佬二十一年 转发微博
第410条转发：喵卷家的喵星人 meme说中文好可爱wwwwkwiiiii/谢谢meme呜呜呜呜呜呜呜呜一己之力把我拉坑到整个团呜呜呜救命🆘 🆘 🆘 全员上手に踊る呜呜呜呜呜呜呜呜全员业务能力都超级强
第411条转发：某私法制裁狗 草拟大坝真的笑得想死 中文说得不错下次别说了
第412条转发：xxxx-yyhbv 再听听meme的中文消消火……希望晚上做梦能梦到开心点的。
第413条转发：白T阿扎西 我赞成！把烟花吻拍出来！//@想打边炉:你们消失的初恋能不能拍到天荒地老——
第414条转发：misa202005 Repost
https://m.weibo.cn/api/statuses/repostTimeline?id=4709336482318969&page=51
第415条转发：CMWLLA 太可爱了呜呜
第416条转发：噢好知道了 我的好大儿 想看你俩摩多摩多
第417条转发：-蒼桥- 笑得想死…
第418条转发：Barbarabwx 转发微博
第419条转发：爱吃天妇罗的兔兔 专和烧普都get了开心
第420条转发：irriss__ 我靠？可以啊
第421条转发：戏精小年轻 转发微博
第422条转发：也可靠我创做浮现 哈哈哈哈哈哈哈哈哈哈哈哈在被窝里笑不活了
https://m.weibo.cn/ap

https://m.weibo.cn/api/statuses/repostTimeline?id=4709336482318969&page=71
第589条转发：鸭洗啦你 😂😂😂😂😂😂
第590条转发：大大大大番茄哇 有一说一没字幕也能听懂，虽然口音真的很搞笑。。《消失的初恋》第二季！第二季！第二季！memi！memi！memi！赛高！
第591条转发：北極星_mwx 转发
第592条转发：AigeTea 这个口音比泰国人还魔性哈哈哈哈哈哈哈哈哈哈哈哈哈哈笑不活了
第593条转发：窗户在淮海路吃炸猪排 笑死，莲哥百万直拍有了
第594条转发：o泡小啵- 第二季！第二季！第二季！第二季！第二季！第二季！第二季！第二季！第二季！第二季！[春游家族][春游家族][春游家族][春游家族][春游家族][春游家族][春游家族][春游家族]//@吹散云朵拥抱我:请一定要和michi一起拍到天荒地老//@想打边炉:你们消失的初恋能不能拍到天荒地老——
第595条转发：Bigflower小羊 回复@从前有个养乐多:河南拔智齿//@从前有个养乐多:消初第二季gkd📣
第596条转发：50watt_smile 干了看了
第597条转发：第六感杀手 睡前再看一遍(^q^)
https://m.weibo.cn/api/statuses/repostTimeline?id=4709336482318969&page=72
第598条转发：麻吉今天喝酒了吗 meme  就咱中文还是有进步的
第599条转发：励志做大官 转发微博
第600条转发：飞丞丞丞丞橙汁儿 宝儿晚安😘😴💤
第601条转发：网瘾小仙女ZzQ 哈哈哈哈哈中文比英文强多了，虽然还是有几句听不懂
第602条转发：啊你猜呀 转发微博
第603条转发：VAnYIGo 谁的百万直拍，めめ的百万直拍[开学季][开学季][开学季]
第604条转发：Ericadengsiyi 转发微博
第605条转发：AchilleLeiste·高考必上600分 meme私はあなたが好きです
第606条转发：一只桑啊 转发微博
https://m.weibo.cn/api/statuses/repostTimeline?id=4709336482318969&page=73
第607条转发：搞钱v 我都要看一百次了！！！//@莲理枝AI:中文视频 好有诚

https://m.weibo.cn/api/statuses/repostTimeline?id=4709336482318969&page=95
第786条转发：DTlove790410 轉發微博
第787条转发：清衫er 百万直拍🥰
第788条转发：翦翦-ving 一时竟听不出你在说中文
第789条转发：放羊的小Fu 太可爱了哈哈哈哈 说完一句伸一下头//@莲理枝AI:中文视频 好有诚意的meme！ 好有诚意的杰尼斯[春游家族]抽个鸟人给我翻译！！！！
第790条转发：哈酱同学就是我 努力说中文的めめ太棒了吧
第791条转发：小顾小顾咕咕咕 好好好好——好可爱！！！！//@目黑区今日播报:#めめ有话说# 🖤 中文特别版请大家查收。 相信大家一定可以从这个视频中感受到他的真诚。 今天也是超爱目黑人士 今后也请继续在追逐梦想的道路上大步向前， 永远做你自己✨ 中国目黑区住民期待着和你相聚的那一天💫
第792条转发：阿清今天休息吗 可可爱爱w
第793条转发：M3djed 掌握财富密码的小明
第794条转发：荣耀的背后刻着一到孤独 转发微博
https://m.weibo.cn/api/statuses/repostTimeline?id=4709336482318969&page=96
第795条转发：不死川- 中文学得好好！
第796条转发：VICKRISNKRYS 速速安排消初第二季 大学生活搞起来！
第797条转发：聪明鸭鸭秘密总部 转发微博
第798条转发：五夏五 目黑莲 我的神
第799条转发：牛乳味猫头鹰 啧啧啧哎哎哎//@冰镇核桃任:什么叫先喜爱消失的初恋再喜欢snowman 你怎么肥事啊me@糖醋理你
第800条转发：花前月下的亮晶晶 给莲带带人气
第801条转发：KYRAxxxx ？
第802条转发：青箬笠共一碗杂豆粥 めめ五单大卖！
第803条转发：凯丽鸡 笑死我，听力考试
https://m.weibo.cn/api/statuses/repostTimeline?id=4709336482318969&page=97
第804条转发：晴子普达 都去给我看！都去给我听！
第805条转发：凌罗卜 超喜欢meme！[开学季]
第806条转发：·几点睡喔· [开学季]//@莲理枝AI:中文视频 好有诚意的meme！ 好有诚意的杰尼斯[春游家族]抽个

...

https://m.weibo.cn/api/statuses/repostTimeline?id=4709336482318969&page=229
第2029条转发：力丸要嫁给我啦 //@一般般肉:《消えた初恋》、私たちは全部見ました、目黒蓮君の演技はすばらしすぎてね!《snow man 5th》が大ヒットを祝います!《消えた初恋2》はありますか?あるなら、私たちはぜひとても楽しみです!今年はまだ1か月残っている、私たちは目黒蓮君に楽しくて幸せで元気に過ごすことを望みます。
第2030条转发：Andrea_xuanxuan 虽然听meme中文还要翻译，但是，哥你好帅uuu，消恋超好看，我爱直球井田和笨蛋meme
第2031条转发：力丸要嫁给我啦 //@一般般肉:《消えた初恋》、私たちは全部見ました、目黒蓮君の演技はすばらしすぎてね!《snow man 5th》が大ヒットを祝います!《消えた初恋2》はありますか?あるなら、私たちはぜひとても楽しみです!今年はまだ1か月残っている、私たちは目黒蓮君に楽しくて幸せで元気に過ごすことを望みます。
第2032条转发：力丸要嫁给我啦 //@一般般肉:《消えた初恋》、私たちは全部見ました、目黒蓮君の演技はすばらしすぎてね!《snow man 5th》が大ヒットを祝います!《消えた初恋2》はありますか?あるなら、私たちはぜひとても楽しみです!今年はまだ1か月残っている、私たちは目黒蓮君に楽しくて幸せで元気に過ごすことを望みます。
第2033条转发：力丸要嫁给我啦 //@一般般肉:《消えた初恋》、私たちは全部見ました、目黒蓮君の演技はすばらしすぎてね!《snow man 5th》が大ヒットを祝います!《消えた初恋2》はありますか?あるなら、私たちはぜひとても楽しみです!今年はまだ1か月残っている、私たちは目黒蓮君に楽しくて幸せで元気に過ごすことを望みます。
第2034条转发：力丸要嫁给我啦 //@一般般肉:《消えた初恋》、私たちは全部見ました、目黒蓮君の演技はすばらしすぎてね!《snow man 5th》が大ヒットを祝います!《消えた初恋2》はありますか?あるなら、私たちはぜひとても楽しみです!今年はまだ1か月残っている、私たちは目黒蓮君に楽しくて幸せで元気に過ごすことを望みます。
第2035条转发：网络社会正义维护员 为什么发语

https://m.weibo.cn/api/statuses/repostTimeline?id=4709336482318969&page=235
第2088条转发：力丸要嫁给我啦 //@一般般肉:《消えた初恋》、私たちは全部見ました、目黒蓮君の演技はすばらしすぎてね!《snow man 5th》が大ヒットを祝います!《消えた初恋2》はありますか?あるなら、私たちはぜひとても楽しみです!今年はまだ1か月残っている、私たちは目黒蓮君に楽しくて幸せで元気に過ごすことを望みます。
第2089条转发：力丸要嫁给我啦 //@一般般肉:《消えた初恋》、私たちは全部見ました、目黒蓮君の演技はすばらしすぎてね!《snow man 5th》が大ヒットを祝います!《消えた初恋2》はありますか?あるなら、私たちはぜひとても楽しみです!今年はまだ1か月残っている、私たちは目黒蓮君に楽しくて幸せで元気に過ごすことを望みます。
第2090条转发：力丸要嫁给我啦 //@一般般肉:《消えた初恋》、私たちは全部見ました、目黒蓮君の演技はすばらしすぎてね!《snow man 5th》が大ヒットを祝います!《消えた初恋2》はありますか?あるなら、私たちはぜひとても楽しみです!今年はまだ1か月残っている、私たちは目黒蓮君に楽しくて幸せで元気に過ごすことを望みます。
第2091条转发：力丸要嫁给我啦 //@一般般肉:《消えた初恋》、私たちは全部見ました、目黒蓮君の演技はすばらしすぎてね!《snow man 5th》が大ヒットを祝います!《消えた初恋2》はありますか?あるなら、私たちはぜひとても楽しみです!今年はまだ1か月残っている、私たちは目黒蓮君に楽しくて幸せで元気に過ごすことを望みます。
第2092条转发：嗳偶练啊 转发微博
第2093条转发：力丸要嫁给我啦 //@一般般肉:《消えた初恋》、私たちは全部見ました、目黒蓮君の演技はすばらしすぎてね!《snow man 5th》が大ヒットを祝います!《消えた初恋2》はありますか?あるなら、私たちはぜひとても楽しみです!今年はまだ1か月残っている、私たちは目黒蓮君に楽しくて幸せで元気に過ごすことを望みます。
第2094条转发：力丸要嫁给我啦 //@一般般肉:《消えた初恋》、私たちは全部見ました、目黒蓮君の演技はすばらしすぎてね!《snow man 5th》

https://m.weibo.cn/api/statuses/repostTimeline?id=4709336482318969&page=244
第2173条转发：英子76697 请拍消失的初恋第二季//@_柒夏:Love you both  评论配图
第2174条转发：陽光宅女_小铁锤丿 嗯！为啥你耳朵也能显微//@Togi_可以这很三赖子 :@陽光宅女_小铁锤丿 我听起来最后最后半秒掐了的声音是skm没错吧wwwwwwww
第2175条转发：永远心动w 杰尼斯你太会了 
第2176条转发：麦记在炸薯条 哈哈哈哈哈哈哈
第2177条转发：是谁重要不重要 喜欢消失的初恋 喜欢meme 和michi 爱你们
第2178条转发：流星妹子-桂在庚随 虽然听不懂，但是很有诚意的样子。
第2179条转发：Togi_可以这很三赖子 @陽光宅女_小铁锤丿 我听起来最后最后半秒掐了的声音是skm没错吧wwwwwwww
第2180条转发：英子76697 不要笑我莲军，严肃点，他值得表扬
第2181条转发：RyoheyRyohei 轉發微博
第2182条转发：馨儀好困哦 莲哥你确实努力了哈哈哈哈哈哈哈哈
https://m.weibo.cn/api/statuses/repostTimeline?id=4709336482318969&page=245
第2183条转发：是小哈本人 再看一遍还是觉得好可爱最后一本正经努力说好的样子好可爱
第2184条转发：钱高端高端 
第2185条转发：圆月气泡 四个瑞大岀，，meme英文发音也太好玩了吧！今天第一份快乐！
第2186条转发：自动贩卖思念机 杰尼斯 雪姨 welcome to内娱 cpf就要骑脸少一边看不起做数据一边做数据
第2187条转发：昏昏欲睡zz 啊啊啊啊啊啊啊啊啊啊啊哥来中国拍戏啊啊啊啊啊啊啊啊啊啊啊啊啊啊啊啊啊啊
第2188条转发：·白日夢探險家· //@莲理枝AI:中文视频 好有诚意的meme！ 好有诚意的杰尼斯[春游家族]抽个鸟人给我翻译！！！！
第2189条转发：等我一fer钟 
第2190条转发：想成为纯欲小男孩 第二季！第二季！
https://m.weibo.cn/api/statuses/repostTimeline?id=4709336482318969&page=246
第2191条转发：瑞丁监狱C3

https://m.weibo.cn/api/statuses/repostTimeline?id=4709336482318969&page=263
第2343条转发：卧龙居士何氏璧 剧看了，嗨挺上头
第2344条转发：芒果荔枝布丁 哇
第2345条转发：Blade_Runner_777 中文很认真啊meme！哈哈哈哈加油[春游家族]
第2346条转发：笑笑鸭呀压 喜欢你真是太棒！了
第2347条转发：笑笑鸭呀压 转发微博
第2348条转发：饭饭onikiri 👀看一个努力的帅哥
第2349条转发：奶牛爱喝草莓牛奶 转发微博
第2350条转发：淡定淡定不要生气 [开学季][开学季][开学季]
第2351条转发：摔倒求堂本二爷亲亲的红豆 好可爱
https://m.weibo.cn/api/statuses/repostTimeline?id=4709336482318969&page=264
第2352条转发：yhapril 
第2353条转发：豆花糕_ 第二季[苦涩][苦涩][苦涩]
第2354条转发：Sophiarains meme好帅好可爱[开学季]
第2355条转发：未未末笙 转发微博
第2356条转发：仙人掌豚鼠 在我是林青霞之后我们要更新杰人说中文了！
第2357条转发：青龙要努力赚钱钱 每天的快乐
第2358条转发：爱巫瓜子的朱朱 好认真在说中文哈哈
第2359条转发：非非兔兔 背景音还有skm的笑声叭（？）哈哈哈哈
第2360条转发：小星星亮晶晶uu 宝贝
https://m.weibo.cn/api/statuses/repostTimeline?id=4709336482318969&page=265
第2361条转发：夏日不限糖 第二季gkd//@快来恰草莓:第二季gkd//@莲理枝AI:中文视频 好有诚意的meme！ 好有诚意的杰尼斯[春游家族]抽个鸟人给我翻译！！！！
第2362条转发：芋头小狗汪 赢麻了//@莲理枝AI:中文视频 好有诚意的meme！ 好有诚意的杰尼斯[春游家族]抽个鸟人给我翻译！！！！
第2363条转发：单字族 
第2364条转发：innocentbottle 才看到这条，我怎么又在哭
第2365条转发：菏泽野牡丹 鸟人🐮哇 评论区沦陷……
第2366条转发：煌黑鱼子酱hq 中文越说越好啦！
第2367条转发：张小念哦

KeyboardInterrupt: 